In [1]:
import d3rlpy
import d4rl
import gym
import h5py
import numpy as np
from d3rlpy.dataset import MDPDataset
from sklearn.model_selection import train_test_split
import gym
from gym.wrappers import TransformObservation
import encoders
import os

No module named 'mjrl'
No module named 'flow'
No module named 'carla'
pybullet build time: May 20 2022 19:44:17
/home/yasin/.local/share/virtualenvs/symmetry-cs285-U_vq4NrE/lib/python3.8/site-packages/pybullet_envs/env_bases.py:8: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import parse_version
/home/yasin/.local/share/virtualenvs/symmetry-cs285-U_vq4NrE/lib/python3.8/site-packages/pkg_resources/__init__.py:2871: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('mpl_toolkits')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)


In [2]:
print(gym.version.VERSION)

0.23.1


In [3]:
seed = 1
d3rlpy.seed(seed)
use_gpu = True
# prepare environment
#env = gym.make("InvertedPendulum-v2")
#eval_env = gym.make("InvertedPendulum-v2")
env = gym.make("Walker2d-v2")
eval_env = gym.make("Walker2d-v2")
env.reset(seed=seed)
eval_env.reset(seed=seed)

/home/yasin/.local/share/virtualenvs/symmetry-cs285-U_vq4NrE/lib/python3.8/site-packages/gym/envs/registration.py:505: UserWarning: WARN: The environment Walker2d-v2 is out of date. You should consider upgrading to version `v3` with the environment ID `Walker2d-v3`.
  logger.warn(


array([ 1.25450464e+00, -3.55840387e-03,  4.48649447e-03, -1.88168548e-03,
       -7.66735510e-04,  3.27702594e-03, -9.08008636e-04,  4.95936877e-04,
       -4.72440887e-03,  2.53513109e-03,  3.81433132e-04, -1.70268284e-03,
        2.88428703e-03, -1.96805171e-03, -4.65021105e-04, -3.65958303e-03,
       -9.68870136e-04])

## Get Dataset

In [37]:
def read_hdf5_to_dict(file_name):
    data = {}

    def recursive_read(group, prefix=''):
        for key in group.keys():
            if key not in ['observations', 'actions', 'terminals', 'rewards', 'timeouts']:
                continue
            item = group[key]
            if isinstance(item, h5py.Dataset):
                # This is a dataset
                data[prefix + key] = np.array(item)
            elif isinstance(item, h5py.Group):
                # This is a group, so recurse into it
                new_prefix = prefix + key + '/'
                recursive_read(item, new_prefix)

    with h5py.File(file_name, 'r') as f:
        recursive_read(f)

    return data
    
file_path = 'd3rlpy_data/walker2d_expert-v2.hdf5'  # Replace with the path to your HDF5 file
data_dict = read_hdf5_to_dict(file_path)

In [38]:
# Example transformation matrices
m = data_dict['observations'].shape[1]
n = data_dict['actions'].shape[1]
P_s = np.zeros((m,m))  # Replace with your P_s matrix
P_s
P_a = np.zeros((n,n))  # Replace with your P_a matrix

In [39]:
def create_permutation_matrix(size, source_indices, target_indices):
    """
    Create a permutation matrix of given size that permutes rows from source_indices to target_indices.

    :param size: Size of the square permutation matrix
    :param source_indices: List of indices to be permuted
    :param target_indices: List of target indices where source indices should be moved
    :return: Permutation matrix of size 'size x size'
    """
    if len(source_indices) != len(target_indices):
        raise ValueError("Source and target indices lists must be of the same length")

    # Create an identity matrix
    perm_matrix = np.identity(size)

    # Apply the permutations
    for src, tgt in zip(source_indices, target_indices):
        perm_matrix[[src, tgt]] = perm_matrix[[tgt, src]]

    return perm_matrix

In [40]:
m = data_dict['observations'].shape[1]
n = data_dict['actions'].shape[1]

P_s = create_permutation_matrix(m, [2, 3, 4, 11, 12, 13], [5, 6, 7, 14, 15, 16])
P_a = create_permutation_matrix(n, [0, 1, 2], [3, 4, 5])

In [41]:
P_s, P_a

(array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
         0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
         0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
         0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
         0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.

In [42]:
def augment_data(data_dict, P_s, P_a):
    # Vectorized transformation
    transformed_observations = np.dot(data_dict['observations'], P_s.T)  # Transpose if necessary
    transformed_actions = np.dot(data_dict['actions'], P_a.T)            # Transpose if necessary
    
    data_dict['observations'] = np.concatenate((data_dict['observations'], transformed_observations), axis=0)
    data_dict['actions'] = np.concatenate((data_dict['actions'], transformed_actions), axis=0)
    data_dict['terminals'] = np.concatenate((data_dict['terminals'], data_dict['terminals']))
    data_dict['rewards'] = np.concatenate((data_dict['rewards'], data_dict['rewards']))
    data_dict['timeouts'] = np.concatenate((data_dict['timeouts'], data_dict['timeouts']))

In [43]:
augment_data(data_dict, P_s, P_a)

In [44]:
dataset = MDPDataset(data_dict['observations'], data_dict['actions'], data_dict['rewards'], np.logical_or(data_dict['terminals'], data_dict['timeouts']))
del data_dict

## Training

In [31]:
train_episodes, test_episodes = train_test_split(dataset, random_state=seed, train_size=0.1)

In [32]:
#encoder_factory = encoders.SymmetryEncoderFactory(project=inverted_pendulum_project, projection_size=projection_size)
#dynamics = d3rlpy.dynamics.ProbabilisticEnsembleDynamics(learning_rate=1e-4, use_gpu=True, state_encoder_factory=encoder_factory)
dynamics = d3rlpy.dynamics.ProbabilisticEnsembleDynamics(learning_rate=1e-4, use_gpu=True) # Baseline

In [ ]:
# same as algorithms
dynamics.fit(train_episodes,
             eval_episodes=test_episodes,
             n_epochs=100,
             scorers={
                'observation_error': d3rlpy.metrics.scorer.dynamics_observation_prediction_error_scorer,
                'reward_error': d3rlpy.metrics.scorer.dynamics_reward_prediction_error_scorer,
                'variance': d3rlpy.metrics.scorer.dynamics_prediction_variance_scorer,
             },
            tensorboard_dir='tensorboard_logs/dynamics',
            experiment_name='SeparateStateAndRewardEncoders')

2023-11-14 21:29:08 [debug    ] RoundIterator is selected.
2023-11-14 21:29:08 [info     ] Directory is created at d3rlpy_logs/SeparateStateAndRewardEncoders_20231114212908
2023-11-14 21:29:08 [debug    ] Building models...
2023-11-14 21:29:08 [debug    ] Models have been built.
2023-11-14 21:29:08 [info     ] Parameters are saved to d3rlpy_logs/SeparateStateAndRewardEncoders_20231114212908/params.json params={'action_scaler': None, 'batch_size': 100, 'discrete_action': False, 'gamma': 1.0, 'generated_maxlen': 100000, 'learning_rate': 0.0001, 'n_ensembles': 5, 'n_frames': 1, 'n_steps': 1, 'optim_factory': {'optim_cls': 'Adam', 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight_decay': 0.0001, 'amsgrad': False}, 'real_ratio': 1.0, 'reward_encoder_factory': {'type': 'default', 'params': {'activation': 'relu', 'use_batch_norm': False, 'dropout_rate': None}}, 'reward_scaler': None, 'scaler': None, 'state_encoder_factory': {'type': 'default', 'params': {'activation': 'relu', 'use_batch_norm': Fal

Epoch 1/100:   0%|          | 0/992 [00:00<?, ?it/s]

2023-11-14 21:29:57 [info     ] SeparateStateAndRewardEncoders_20231114212908: epoch=1 step=992 epoch=1 metrics={'time_sample_batch': 0.00025271119609955817, 'time_algorithm_update': 0.02303843056001971, 'loss': -39.001211083471595, 'time_step': 0.023374602919624697, 'observation_error': 43.4004609201577, 'reward_error': 0.016453990222475432, 'variance': 22.547224731700684} step=992
2023-11-14 21:29:57 [info     ] Model parameters are saved to d3rlpy_logs/SeparateStateAndRewardEncoders_20231114212908/model_992.pt


Epoch 2/100:   0%|          | 0/992 [00:00<?, ?it/s]

2023-11-14 21:30:41 [info     ] SeparateStateAndRewardEncoders_20231114212908: epoch=2 step=1984 epoch=2 metrics={'time_sample_batch': 0.00023783887586285992, 'time_algorithm_update': 0.022105127332671996, 'loss': -61.47404948021135, 'time_step': 0.02241803249043803, 'observation_error': 45.788177242730626, 'reward_error': 0.01016925438169652, 'variance': 25.64507883825302} step=1984
2023-11-14 21:30:41 [info     ] Model parameters are saved to d3rlpy_logs/SeparateStateAndRewardEncoders_20231114212908/model_1984.pt


...

Epoch 98/100:   0%|          | 0/992 [00:00<?, ?it/s]

2023-11-14 22:37:11 [info     ] SeparateStateAndRewardEncoders_20231114212908: epoch=98 step=97216 epoch=98 metrics={'time_sample_batch': 0.00021092497533367526, 'time_algorithm_update': 0.01790240599263099, 'loss': -125.14057612226856, 'time_step': 0.018171135215990005, 'observation_error': 34.45343410092868, 'reward_error': 0.001325730130646736, 'variance': 22.157576461240684} step=97216
2023-11-14 22:37:11 [info     ] Model parameters are saved to d3rlpy_logs/SeparateStateAndRewardEncoders_20231114212908/model_97216.pt


Epoch 99/100:   0%|          | 0/992 [00:00<?, ?it/s]

2023-11-14 22:37:50 [info     ] SeparateStateAndRewardEncoders_20231114212908: epoch=99 step=98208 epoch=99 metrics={'time_sample_batch': 0.00020923080944245862, 'time_algorithm_update': 0.018125137975139004, 'loss': -124.2604928420436, 'time_step': 0.018391027806266662, 'observation_error': 35.199419645193714, 'reward_error': 0.001197132637240125, 'variance': 23.095723561930722} step=98208
2023-11-14 22:37:50 [info     ] Model parameters are saved to d3rlpy_logs/SeparateStateAndRewardEncoders_20231114212908/model_98208.pt


Epoch 100/100:   0%|          | 0/992 [00:00<?, ?it/s]